# Submitting a large batch of jobs

Often, you're going to want a big dataset that will take a while to render -- and you probably don't want to keep a notebook kernel going overnight. This notebook writes the submitted job IDs to disk, which allows you to close the notebook and re-open it at a later time to query the jobs' statuses. You can then choose to download data as individual jobs complete, or wait until the entire batch has completed. 

Specifically, this notebook shows how to:
  1. Define a distribution of parameters for the batch submission.
  2. Submit many jobs in batch mode.
  3. Query job status in non-blocking mode.
  4. Visualize dataset statistics + filter for specific properties.

----- 

__Note__: The demonstration here was downsampled for easier viewing in the notebook, so the outputs in the cells will reflect small batch jobs. You can generate batch jobs with many more videos that have higher resolution, better frame rates, and more reps.

-----

In [ ]:
import random
from pprint import pprint
import infinity_tools.visionfit.api as api
from infinity_tools.visionfit.vis import visualize_job_params
from infinity_tools.visionfit.vis import summarize_batch_results_as_dataframe, visualize_batch_results

## Constants

In [ ]:
TOKEN = "TOKEN"
SERVER = "https://api.toinfinity.ai/"
GENERATOR = "visionfit-v0.4.0"
OUT_DIR = "tmp/"

In [ ]:
sesh = api.VisionFitSession(token=TOKEN, generator=GENERATOR, server=SERVER)

### Check API Usage

In [ ]:
# Get your usage stats for the last N days (30 in the example below).
usage_data_last_30_days = sesh.get_usage_stats_last_n_days(n_days=30)
print("Usage in the last 30 days")
pprint(usage_data_last_30_days)

# Use datetime and a lower-level API for a more precise/custom range of usage stats.
from datetime import datetime
from infinity_core.api import get_usage_datetime_range
start_time = datetime.fromisoformat("2022-07-15")
end_time = datetime.fromisoformat("2022-08-20")
usage_data_late_summer_2022 = get_usage_datetime_range(token=TOKEN, server=SERVER, start_time=start_time, end_time=end_time)
print("\nUsage in late summer 2022")
pprint(usage_data_late_summer_2022.json())

### Define distribution of parameters for the batch submission 

In this example, we will show how to create a large dataset of arm raise recordings with the following constraints:
  - All videos will contain a single type of exercise (`exercise`, ARM_RAISE-DUMBBELL) 
  - All videos have 4 total reps (`num_reps`, 4)
  - The resulting distribution of gender will be evenly split (`gender`, 50% male and 50% female)
  - Each scene will be well-lit but not over-saturated (`lighting_power`, ~300 units)
  - The camera will be placed between floor and table height (`camera_height`, ~1.2 meters)
  - The resulting videos will have a low frame rate (`frame_rate`, 6 fps)
  - The resulting videos will have a resolution of 256x256 pixels (`image_height` and `image_width`, 256 pixels x 256 pixels)
  
**Users can update the specification of `job_params` to fit their exact dataset needs.**

In [ ]:
num_jobs = 1000
batch_name = "uppercut left large job demo" # change to a description of your dataset.

# Set some initial parameters.
job_params = [
        sesh.sample_input(
            exercise = "UPPERCUT-LEFT",
            num_reps = 4,
            lighting_power = float(random.gauss(300.0, 20.0)),
            camera_height = float(random.gauss(1.2, 0.2)),
            frame_rate=6,
            image_height=256,
            image_width=256
        ) for _ in range(num_jobs)
    ]

Before submitting our full video jobs to the API, we can visualize the distributions of our parameters:

In [ ]:
visualize_job_params(job_params)

### Submit many jobs in batch mode.

After verifying our parameters are in the correct range, we can submit a large batch of jobs to the API.

High-resolution videos can take some time to render (~20 minutes per video), so this part is **non-blocking**. That is, you can run this cell and then shut down your notebook if you like. The job ids are saved to your local disk, so you can pick right up in the next section when you come back.

**Note**: We don't actually want to submit a job for 1000 videos for this demo notebook, so the next cell will just take the first 5. **Delete the next cell when you're ready to generate videos at scale.**

In [ ]:
job_params = job_params[:5] #comment out or delete this cell for the full jobs array.

In [ ]:
batch = sesh.submit(
    job_params=job_params,
    is_preview=False,
    batch_name=batch_name,
)

### Reconstituting a batch later on

You can reconstitute a submitted batch locally from the Infinity cloud API. Examples of when you would want to do this: (a) you come back to check on a large batch in a fresh notebook instance at a later date, (b) your notebook instance crashes or you exited it out, or (c) you want to revisit a batch from a long time ago.

You can always view your batches by logging in to the [Infinity User Portal](https://api.toinfinity.ai/admin/api/batch/). This will give you information about previously submitted batches and their individual jobs.

To locally reconstitute a batch, just grab the `batch_id` from the batch you are interested in from the User Portal.

In [ ]:
# Here, we will get the `batch_id` from the `batch` instance in local memory. But you can easily get the
# `batch_id` from the Infinity User Portal.
batch_id = batch.batch_id

### Query job status in non-blocking mode.

We next show how to check on the status of a specific batch (potentially after closing and restarting this notebook). You only need to provide a path to the associated folder. Note that you can choose to move onto the next cell and download data before all jobs have successfully completed.

In [ ]:
# Reconstitute our batch process state (replace `batch_id` with the ID for your desired batch).
batch = sesh.batch_from_api(batch_id=batch_id)

# Poll the server to see the status of our batch job.
print(f"{batch.num_jobs - batch.get_num_jobs_remaining()}/{batch.num_jobs} submitted jobs have completed")

completed_jobs = batch.get_completed_jobs()
valid_jobs = batch.get_valid_completed_jobs()

num_submitted = batch.num_jobs
num_completed = len(completed_jobs)
num_in_progress = num_submitted - num_completed
num_successful = len(valid_jobs)
num_failed = num_completed - num_successful

print(f"{num_successful}/{num_completed} completed jobs have a valid URL.")

#### Download completed jobs

Using the cell from above, we can download the completed jobs. We will only download the completed jobs that have a valid URL.

In [ ]:
batch_path = OUT_DIR + batch_name
_download_ok = batch.download(path=batch_path)

### Visualize dataset statistics + filter for specific properties

Finally, we compile some of the metadata and all of the job parameters that were submitted into a dataframe. This allows us to see the distribution of the resulting dataset. In addition, we can query the dataset for specific properties, which allows us to curate a desired training set for a given ML application.

In [ ]:
import pandas as pd
pd.options.display.max_columns = None

df = summarize_batch_results_as_dataframe(batch_path)
df.round(2).query('avg_percent_occlusion < 4')

In [ ]:
visualize_batch_results(batch_path)